# 学習用動画データ(mp4)とアノテーションデータ(json)を読みこみ、指定した縦横のサイズにリサイズする。

In [2]:
import cv2
import json

In [3]:
# リサイズしたい動画とアノテーションデータのパスを指定する。
train_movie_path = "./train_videos/train_00.mp4"
train_annotations_path = "./train_annotations/train_00.json"
# リサイズ後の動画とアノテーションデータの保存先のパスを指定する。
train_movie_resized_path = "./train_videos_resized/train_00.mp4"
train_annotations_resized_path = "./train_annotations_resized/train_00.json"

In [4]:
# 学習用動画データのロード
video = cv2.VideoCapture(train_movie_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
print(size)

resized_width = 416 # リサイズ後の横幅
resized_height = 416 # リサイズ後の縦幅
resized_size = (resized_width, resized_height)
scale_2_resize_width = resized_width / width # リサイズ後とリサイズ前の横幅の比率
scale_2_resize_height = resized_height / height # リサイズ後とリサイズ前の縦幅の比率
print(scale_2_resize_width)
print(scale_2_resize_height)

(1936, 1216)
0.21487603305785125
0.34210526315789475


In [4]:
# 動画をリサイズして保存する。
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter(train_movie_resized_path, fmt, frame_rate, resized_size)
while True:
    ret, frame = video.read()
    if ret is False:
        break
    frame = cv2.resize(frame, dsize=resized_size)
    writer.write(frame)
writer.release()
video.release()
cv2.destroyAllWindows()

In [5]:
# 動画のリサイズに合わせてアノテーションデータを修正します。

# 学習用動画データのアノテーションデータのオープン
with open(train_annotations_path, 'r') as f:
    train_annotations_json = json.load(f) # 辞書型に変換

# 1フレームずつ処理します。
for sequence in range(len(train_annotations_json['sequence'])):
    # 1フレームに含まれるデータをカテゴリずつ処理します。
    for category in train_annotations_json['sequence'][sequence].keys():
        # 1フレームの1カテゴリに対して、真値のデータは複数あるので一つずつ処理します。
        for index, rect_info in enumerate(train_annotations_json['sequence'][sequence][category]):
            box2d = rect_info['box2d']
            train_annotations_json['sequence'][sequence][category][index]['box2d'][0] = int(box2d[0] * scale_2_resize_width)
            train_annotations_json['sequence'][sequence][category][index]['box2d'][1] = int(box2d[1] * scale_2_resize_height)
            train_annotations_json['sequence'][sequence][category][index]['box2d'][2] = int(box2d[2] * scale_2_resize_width)
            train_annotations_json['sequence'][sequence][category][index]['box2d'][3] = int(box2d[3] * scale_2_resize_height)
                                                                                   
with open(train_annotations_resized_path, 'w') as f:
    json.dump(train_annotations_json, f)

## 全トレーニングデータにリサイズ処理を実施します。

In [10]:
# 上記のリサイズ処理はutil.pyにモジュール化してあるので読み込みます。
from util import resize_video
from util import resize_annotation

train_data_num = 25 # 学習用動画データの数

In [6]:
# 全動画データをリサイズ
save_fourcc = "mp4v"
resized_width = 416 # リサイズ後の横幅
resized_height = 416 # リサイズ後の縦幅
for index in range(train_data_num):
    train_video_path = "./train_videos/train_{:02}.mp4".format(index) # 読込先
    train_video_resized_path = "./train_videos_resized/train_{:02}.mp4".format(index) # 保存先
    resize_video(train_video_path, train_video_resized_path, save_fourcc, resized_width, resized_height)

In [9]:
# 全アノテーションデータをリサイズ
width = 1936 # 元動画の横幅
height = 1216 # 元動画の縦幅
scale_2_resize_width = resized_width / width # リサイズ後とリサイズ前の横幅の比率
scale_2_resize_height = resized_height / height # リサイズ後とリサイズ前の縦幅の比率

for index in range(train_data_num):
    train_annotation_path = "./train_annotations/train_{:02}.json".format(index) # 読込先
    train_annotation_resized_path = "./train_annotations_resized/train_{:02}.json".format(index) # 保存先
    resize_annotation(train_annotation_path, train_annotation_resized_path, scale_2_resize_width, scale_2_resize_height)